In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.utils import to_categorical
import numpy as np
import matplotlib.pyplot as plt
import os

print(tf.__version__)
print(keras.__version__)

2.3.0
2.4.0


In [4]:
learning_rate = 0.001
training_epochs = 15
batch_size = 100

# ↓체크포인트 파일을 만들어서 따로 관리
cur_dir = os.getcwd()
ckpt_dir_name = 'checkpoints'
model_dir_name = 'minst_cnn_seq'

checkpoint_dir = os.path.join(cur_dir, ckpt_dir_name, model_dir_name)
os.makedirs(checkpoint_dir, exist_ok=True)

checkpoint_prefix = os.path.join(checkpoint_dir, model_dir_name)

In [14]:
mnist = tf.keras.datasets.mnist

(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

print("before : ", train_images.shape, test_images.shape)
# before :  (60000, 28, 28) (10000, 28, 28) / channel이 빠져있다.
train_images = train_images.astype(np.float32) / 255.
test_images = test_images.astype(np.float32) / 255.

# traing_images와 test_images가 현재 3차원 이므로 4차원으로 변경
train_images = np.expand_dims(train_images, axis = -1) # axis = -1 : 마지막 차원에 하나 추가
test_images = np.expand_dims(test_images, axis = -1)
print("after : ", train_images.shape, test_images.shape)


train_labels = to_categorical(train_labels, 10) # to_categorical : one-hot 인코딩
test_labels = to_categorical(test_labels, 10)

# train_images와 train_labels를 batch_size만큼 잘라서 대입
train_dataset = tf.data.Dataset.from_tensor_slices((train_images,
                                                   train_labels)).shuffle(buffer_size=100000).batch(batch_size)
test_dataset = tf.data.Dataset.from_tensor_slices((test_images,
                                                   test_labels)).batch(batch_size)


before :  (60000, 28, 28) (10000, 28, 28)
after :  (60000, 28, 28, 1) (10000, 28, 28, 1)


# Build a Neural Network Model
- Sequential API

In [15]:
def create_model():
    model = keras.Sequential() # Sequential 을 사용하겠다고 선언해주어야 함
    # Sequential : add를 통해 한층 한층 쌓아가는 것
    # 처음 add가 처음 layer 두번째 add가 두번째 layer
    model.add(keras.layers.Conv2D(filters=32, kernel_size=3, activation=tf.nn.relu, padding='SAME', 
                                  input_shape=(28, 28, 1)))
    # default값으로 2x2, stride=2 인 pooling이 들어갔는데 이것을 쓰지않고 padding만 same
    model.add(keras.layers.MaxPool2D(padding='SAME')) 
    model.add(keras.layers.Conv2D(filters=64, kernel_size=3, activation=tf.nn.relu, padding='SAME'))
    model.add(keras.layers.MaxPool2D(padding='SAME'))
    model.add(keras.layers.Conv2D(filters=128, kernel_size=3, activation=tf.nn.relu, padding='SAME'))
    model.add(keras.layers.MaxPool2D(padding='SAME'))
    
    model.add(keras.layers.Flatten()) # 앞에서 나왔던 featuer map을 vector형태로 펴줌
    model.add(keras.layers.Dense(256, activation=tf.nn.relu))
    model.add(keras.layers.Dropout(0.4))
    model.add(keras.layers.Dense(10))
    return model

In [16]:
model = create_model()
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 28, 28, 32)        320       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 14, 14, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 14, 14, 64)        18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 7, 7, 64)          0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 7, 7, 128)         73856     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 4, 4, 128)         0         
_________________________________________________________________
flatten (Flatten)            (None, 2048)              0

# Define a Loss Function

In [17]:
def loss_fn(model, images, labels):
    logits = model(images, training=True)
    loss = tf.reduce_mean(tf.keras.losses.categorical_crossentropy(
        y_pred=logits, y_true=labels, from_logits=True))    
    return loss

In [18]:
def grad(model, images, labels):
    with tf.GradientTape() as tape:
        loss = loss_fn(model, images, labels)
    return tape.gradient(loss, model.variables)

In [19]:
optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)

In [20]:
def evaluate(model, images, labels):
    logits = model(images, training=False)
    correct_prediction = tf.equal(tf.argmax(logits, 1), tf.argmax(labels, 1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
    return accuracy

In [21]:
checkpoint = tf.train.Checkpoint(cnn=model)

In [22]:
def train(model, images, labels):
    grads = grad(model, images, labels)
    optimizer.apply_gradients(zip(grads, model.trainable_variables))

In [23]:
print('Learning started. It takes sometime.')
for epoch in range(training_epochs):
    avg_loss = 0.
    avg_train_acc = 0.
    avg_test_acc = 0.
    train_step = 0
    test_step = 0    
    
    for images, labels in train_dataset:
        train(model, images, labels)
        #grads = grad(model, images, labels)                
        #optimizer.apply_gradients(zip(grads, model.variables))
        loss = loss_fn(model, images, labels)
        acc = evaluate(model, images, labels)
        avg_loss = avg_loss + loss
        avg_train_acc = avg_train_acc + acc
        train_step += 1
    avg_loss = avg_loss / train_step
    avg_train_acc = avg_train_acc / train_step
    
    for images, labels in test_dataset:        
        acc = evaluate(model, images, labels)        
        avg_test_acc = avg_test_acc + acc
        test_step += 1    
    avg_test_acc = avg_test_acc / test_step    

    print('Epoch:', '{}'.format(epoch + 1), 'loss =', '{:.8f}'.format(avg_loss), 
          'train accuracy = ', '{:.4f}'.format(avg_train_acc), 
          'test accuracy = ', '{:.4f}'.format(avg_test_acc))
    
    checkpoint.save(file_prefix=checkpoint_prefix)

print('Learning Finished!')

Learning started. It takes sometime.
Epoch: 1 loss = 0.18206577 train accuracy =  0.9552 test accuracy =  0.9875
Epoch: 2 loss = 0.04645314 train accuracy =  0.9901 test accuracy =  0.9900
Epoch: 3 loss = 0.03087426 train accuracy =  0.9931 test accuracy =  0.9922
Epoch: 4 loss = 0.02308669 train accuracy =  0.9949 test accuracy =  0.9925
Epoch: 5 loss = 0.01812660 train accuracy =  0.9963 test accuracy =  0.9930
Epoch: 6 loss = 0.01582826 train accuracy =  0.9969 test accuracy =  0.9941
Epoch: 7 loss = 0.01321064 train accuracy =  0.9980 test accuracy =  0.9931
Epoch: 8 loss = 0.01011426 train accuracy =  0.9981 test accuracy =  0.9926
Epoch: 9 loss = 0.00916506 train accuracy =  0.9986 test accuracy =  0.9920
Epoch: 10 loss = 0.00840842 train accuracy =  0.9990 test accuracy =  0.9906
Epoch: 11 loss = 0.00611641 train accuracy =  0.9992 test accuracy =  0.9937
Epoch: 12 loss = 0.00646833 train accuracy =  0.9992 test accuracy =  0.9914
Epoch: 13 loss = 0.00614800 train accuracy =  0.